In [4]:
import os
import csv
import re

# Liste des mots à ignorer (stop words) en français et en anglais
stop_words = set([
    'le', 'la', 'les', 'un', 'une', 'des', 'du', 'de', 'au', 'aux', 'ce', 'cette', 'ces',
    'mon', 'ton', 'son', 'notre', 'votre', 'leur', 'et', 'ou', 'mais', 'donc', 'or', 'ni',
    'car', 'à', 'de', 'en', 'sur', 'sous', 'chez', 'dans', 'par', 'pour', 'the', 'a', 'an', 
    'and', 'or', 'but', 'so', 'because', 'of', 'in', 'on', 'at', 'with', 'by', 'for',
    'what', 'is', 'this', 'it', 'that', 'i', 'you', 'he', 'she', 'we', 'they', 'has', 'my', 'as'
])

# Initialiser les variables pour les identifiants
global_word_id = 1
movie_id_map = {}
words_data = []

def clean_text(text):
    """
    Nettoyer le texte en supprimant les mots à ignorer (stop words) et balises HTML.
    """
    text = re.sub(r'<.*?>', '', text)  # Supprimer les balises HTML
    text = re.sub(r"(\w)'(\w)", r"\1'\2", text)  # Conserver les mots avec apostrophe ensemble (comme c'est, l'avion)

    # Utiliser une expression régulière avec le flag re.UNICODE pour capturer les caractères accentués
    words = re.findall(r'\b[\w\'-]+\b', text.lower(), re.UNICODE)

    filtered_words = []
    i = 0
    while i < len(words):
        word = words[i]
        if word not in stop_words:
            # Vérification si l'apostrophe est suivie d'un mot
            if i < len(words) - 1 and re.match(r"\w'", word):
                word = f"{word}{words[i+1]}"
                i += 1  # Sauter le mot suivant
            filtered_words.append(word)
        i += 1

    return filtered_words

def process_srt_file(file_path, movie_title):
    """
    Traiter un fichier .srt et extraire les mots nettoyés des sous-titres.
    """
    global global_word_id
    if not os.path.exists(file_path):
        print(f"Le fichier {file_path} n'existe pas. Ignoré.")
        return

    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            lines = file.readlines()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='ISO-8859-1', errors='ignore') as file:
            lines = file.readlines()

    for line in lines:
        if not re.match(r'^\d+$', line.strip()) and '-->' not in line:  # Ignorer les numéros et les timecodes
            words = clean_text(line.strip())
            for word in words:
                words_data.append({
                    'word_id': global_word_id,
                    'movie_id': movie_id_map[movie_title],
                    'word': word
                })
                global_word_id += 1

def traverse_directories_recursive(base_path):
    """
    Parcourir les répertoires de manière récursive pour trouver et traiter les fichiers .srt.
    """
    if os.path.isdir(base_path):
        for entry in os.listdir(base_path):
            entry_path = os.path.join(base_path, entry)
            traverse_directories_recursive(entry_path)
    elif base_path.endswith('.srt'):
        # Extraire le titre du film depuis le nom du fichier
        file_name = os.path.basename(base_path)
        # Rechercher le titre avant les délimiteurs espace, point, 's' suivi de deux chiffres,
        # "h" suivi de chiffres, "S" suivi de 2 chiffres, ou "g1h" suivi de chiffres
        movie_title_match = re.search(r'^(.*?)(?:\s|\.|s\d{2}|h\d{2}[VFVO]|S\d{2}e\d{2}(e\d{2})?|g1h\d{2}[VFVO])', file_name)
        
        if movie_title_match:
            movie_title = movie_title_match.group(1).strip()
            if movie_title not in movie_id_map:
                movie_id_map[movie_title] = len(movie_id_map) + 1
            process_srt_file(base_path, movie_title)

def write_word_csv(output_file):
    """
    Écrire les mots extraits dans un fichier CSV sans la colonne 'movie_title'.
    """
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['word_id', 'movie_id', 'word'])
        for word_entry in words_data:
            writer.writerow([
                word_entry['word_id'],
                word_entry['movie_id'],
                word_entry['word']
            ])

def write_movie_csv(output_file):
    """
    Écrire l'association entre l'ID de film et le titre de film dans un fichier CSV séparé.
    """
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['movie_id', 'movie_title'])
        for movie_title, movie_id in movie_id_map.items():
            writer.writerow([movie_id, movie_title])

# Exécution principale
base_directory = 'sous-titres'
words_csv = 'words_cleaned.csv'
movies_csv = 'movies_list.csv'

traverse_directories_recursive(base_directory)
write_word_csv(words_csv)
write_movie_csv(movies_csv)
